In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install haversine

In [ ]:
import pandas as pd
import datetime
import time
from haversine import haversine,Unit

In [ ]:
# @title Reading the data
data = pd.read_csv("/content/gdrive/My Drive/Thesis-data/1398-11.csv",low_memory=False)
# data = pd.read_csv("/content/gdrive/My Drive/Thesis-data/1399-03.csv",low_memory=False)

data = data[data['type o']== 's']
data = data[data['type d']== 's']

In [ ]:
#@title Reading the location data
data_location= pd.read_csv("/content/gdrive/My Drive/Thesis-data/location_data1.csv",low_memory=False)
data_location = data_location[data_location['fclass'] == 'railway_station']
data_location = data_location.drop(['name','FID','clusters','line code','fclass'],1)

In [ ]:
# @title Modifing data format
data = data[data['time d'] != 'none']
data = data[data['time d'].isna() != True]

data['date & time D'] = pd.to_datetime(data['Date'] + ' ' + data['time d'])
data['date & time O'] = pd.to_datetime(data['Date'] + ' ' + data['time o'])

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

data['Day_name'] = data['Date'].dt.day_name()

In [ ]:
data_ID = pd.read_csv("/content/gdrive/My Drive/Thesis-data/CH3_F_10000_All(F).csv",low_memory=False)
print(len(data_ID))
ID = data_ID['ID'].unique()
print(len(ID))

In [ ]:
#@title Finding frequent users ID
data1X = data.groupby(['ID']).size().reset_index(name='count')
data2X = data1X[data1X['count'] >= 30]
## List of IDs
lID = data2X['ID'].unique()

In [ ]:
#@title Extracting all data related to those 10000 IDs 
lID_2 = lID[:10000]
data = data[data['ID'].isin(lID_3)]

In [ ]:
#@title Finding activity attributes from trip sequences for each individual
j=[]
a=[]
def attribute_finder(x):

    j.extend(list(x['ID']))
    j2 = len(data)

    i=0
    while i < len(x)-1:
        
        o = x.iloc[i]
        d = x.iloc[i+1]

        lat_lng_d = (data_location[data_location['code']==o['d']]['POINT_Y'].values[0],data_location[data_location['code']==o['d']]['POINT_X'].values[0])
        lat_lng_o = (data_location[data_location['code']==d['o']]['POINT_Y'].values[0],data_location[data_location['code']==d['o']]['POINT_X'].values[0])
        distance = haversine(lat_lng_d,lat_lng_o)

        c = (o['ID'],o['Date'], o['Day_name'], o['d'],o['time d'],(d['date & time O']-o['date & time D']).seconds/3600, distance, (o['d'],d['o']))
        a.append(c)
        i+=1



df = data.groupby(data['ID']).apply(attribute_finder)

In [ ]:
#@title Creating a dataframe of attributes
df2 = pd.DataFrame(a, columns=['ID','Date','Day_name','d','time d','duration','distance','stations'])

## Choosing activities that have less than 3km between origin and destination
df2 = df2[df2['distance'] <= 3]
## Choosing activities that have less than 20 hours duration
df2 = df2[df2['duration'] <20]
## Formating the datasets
df2['time d'] = pd.to_datetime(df2['time d']).dt.hour
df2['duration'] = df2['duration'].astype(int)
df2['duration'] = df2['duration'] + 1
## Saving the datasets
df2.to_csv("/content/gdrive/My Drive/Thesis-data/CH3_F_10000_1398_11_extra10000.csv",index=False)